In [1]:
import tensorflow as tf

n_imputs = 28*28 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10



In [2]:
X = tf.placeholder(tf.float32, shape=(None, n_imputs), name="X")
y = tf.placeholder(tf.int64, shape=(None),name="y")



In [6]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/ np.sqrt(n_imputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
        

In [11]:
import numpy as np

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                          activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                          activation=tf.nn.relu)
    ligits = neuron_layer(hidden2, n_outputs, name="outputs")
    
    

In [12]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(X, n_hidden2, name="hidden2",
                             activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

W0828 11:34:06.788676 140606474753856 deprecation.py:323] From <ipython-input-12-313df860fcfd>:3: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0828 11:34:06.790925 140606474753856 deprecation.py:506] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
with tf.name_scope("loss"):
    #equivalent with applying solfmax then computing cross_entr
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                             logits=logits)
    loss = tf.reduce_mean(xentropy,name="loss")

In [14]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [19]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [24]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [25]:
#Execution Phase
from tensorflow.examples.tutorials.mnist import input_data

In [26]:
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [29]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                           y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, "Models/digists.ckpt")

0 Train accuracy: 0.88 Test accuracy: 0.8949
1 Train accuracy: 0.9 Test accuracy: 0.9073
2 Train accuracy: 0.88 Test accuracy: 0.9159
3 Train accuracy: 0.86 Test accuracy: 0.9222
4 Train accuracy: 0.8 Test accuracy: 0.9266
5 Train accuracy: 0.92 Test accuracy: 0.9307
6 Train accuracy: 0.94 Test accuracy: 0.9349
7 Train accuracy: 0.96 Test accuracy: 0.9376
8 Train accuracy: 0.98 Test accuracy: 0.9398
9 Train accuracy: 0.96 Test accuracy: 0.9405
10 Train accuracy: 0.98 Test accuracy: 0.9436
11 Train accuracy: 0.98 Test accuracy: 0.9449
12 Train accuracy: 0.96 Test accuracy: 0.9474
13 Train accuracy: 1.0 Test accuracy: 0.9484
14 Train accuracy: 0.96 Test accuracy: 0.9509
15 Train accuracy: 0.98 Test accuracy: 0.9514
16 Train accuracy: 0.92 Test accuracy: 0.9538
17 Train accuracy: 0.98 Test accuracy: 0.9542
18 Train accuracy: 0.94 Test accuracy: 0.9539
19 Train accuracy: 0.94 Test accuracy: 0.9569
20 Train accuracy: 0.96 Test accuracy: 0.9572
21 Train accuracy: 0.96 Test accuracy: 0.9584
2

In [ ]:
#make predictions

with tf.Session() as sess:
    saver.restore(sess, "Models/digits.ckpt")
    X_new_scaled=[] #some new image (scaled from 0 to 1)
    Z = logists.eval(feed_dict{ X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)